<h2 style="color:orange;">Lab Assignment 6 - Applications of LLMs</h2>

**AUTHOR NAME : BALBIR SINGH**

**ASU ID :1233870107** 

**FILE CREATION DATE  :3/4/2025**

In [33]:
# Code Cell 1 - Import Required Libraries and Load Data
!pip install s3fs boto3
import s3fs
import boto3
import pandas as pd
import numpy as np
import json
import sagemaker
from sagemaker import get_execution_role
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score


# Load data from S3
s3_uri = "s3://amazon-sagemaker-058264306111-us-east-1-e23504aef6c5/dzd_5l5kah6gnsnq3r/3vronhkhrr0uqv/dev/restaurant_reviews_az.csv"

def load_data(s3_uri):
    df = pd.read_csv(s3_uri)
    return df

df = load_data(s3_uri)
print("Data loaded successfully!")
print(df.head())
print(df.columns)

Data loaded successfully!
                review_id                 user_id             business_id  \
0  IVS7do_HBzroiCiymNdxDg  fdFgZQQYQJeEAshH4lxSfQ  sGy67CpJctjeCWClWqonjA   
1  QP2pSzSqpJTMWOCuUuyXkQ  JBLWSXBTKFvJYYiM-FnCOQ  3w7NRntdQ9h0KwDsksIt5Q   
2  oK0cGYStgDOusZKz9B1qug  2_9fKnXChUjC5xArfF8BLg  OMnPtRGmbY8qH_wIILfYKA   
3  E_ABvFCNVLbfOgRg3Pv1KQ  9MExTQ76GSKhxSWnTS901g  V9XlikTxq0My4gE8LULsjw   
4  Rd222CrrnXkXukR2iWj69g  LPxuausjvDN88uPr-Q4cQA  CA5BOxKRDPGJgdUQ8OUOpw   

   stars  useful  funny  cool  \
0      3       1      1     0   
1      5       1      1     1   
2      5       1      0     0   
3      5       0      0     0   
4      4       1      0     0   

                                                text                 date  
0  OK, the hype about having Hatch chili in your ...  2020-01-27 22:59:06  
1  Pandemic pit stop to have an ice cream.... onl...  2020-04-19 05:33:16  
2  I was lucky enough to go to the soft opening a...  2020-02-29 19:43:44  
3  I've 

In [34]:
# Code Cell 2 - Data Preprocessing
df = df[df['stars'] != 3]  # Remove 3-star reviews
df['Sentiment'] = df['stars'].apply(lambda x: 1 if x >= 4 else 0)

# Randomly selecting 50 positive and 50 negative reviews
positive_reviews = df[df['Sentiment'] == 1].sample(50, random_state=42)
negative_reviews = df[df['Sentiment'] == 0].sample(50, random_state=42)
selected_reviews = pd.concat([positive_reviews, negative_reviews])

# Display dataset statistics
print("Preprocessed Dataset - 50 Positive & 50 Negative Reviews")
print(selected_reviews['Sentiment'].value_counts())
print(selected_reviews.head())

# Save processed data to S3
processed_s3_uri = "amazon-sagemaker-058264306111-us-east-1-e23504aef6c5v"  # Replace with your S3 bucket
selected_reviews.to_csv(processed_s3_uri, index=False)
print("Processed data saved to S3!")

Preprocessed Dataset - 50 Positive & 50 Negative Reviews
Sentiment
1    50
0    50
Name: count, dtype: int64
                    review_id                 user_id             business_id  \
24176  PekPj5ZR4Mrky3FtRpoqXg  T2Qv4ZJ_-fw-aDhc2jEZVg  KyxAss4DMrT_GMzcOLE2yg   
46200  Qy3cpgpNyAYEEreuSJZB_Q  qQ4Ht8TbyC9E77NBz1jIlA  g0OWissnXQgRJVJGYv1mfQ   
12935  5yiSesTG0xL6Lh2L7OyW5A  NKeybkae800m57yQ7U3jtw  L-wv-QK9VoUZX6zUcJwSSw   
860    B4JaHqoRtS0iJOvj1dXg0g  SKkvivmY2b9crEW1NLZHsg  u4P6hqDz6-QG9PR2Pj5KIw   
17966  kJ9ilSRbL2hdF7qBJrzYIg  a_1AD6ACXLSHpztr1k_bTg  LZzDvgfpkd4nI3E4L9wF1w   

       stars  useful  funny  cool  \
24176      4       2      0     1   
46200      4       1      1     1   
12935      5       0      0     0   
860        5       0      0     0   
17966      5       0      0     0   

                                                    text                 date  \
24176  We had the restaurant to ourselves, which was ...  2020-10-27 13:13:27   
46200  When my fami

In [35]:
# Code Cell 3 - Perform Sentiment Analysis Using Zero-Shot Learning
import boto3
import json
from tqdm import tqdm
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

# Create the Bedrock client
client = boto3.client("bedrock-runtime", region_name="us-east-1")

# Define the correct model ID for Claude 3 Sonnet
model_id = 'anthropic.claude-3-sonnet-20240229-v1:0'

# Define the zero-shot prompt template
prompt_template = """
You are an expert sentiment analyzer trained to classify restaurant reviews.
Analyze the following review and classify it as either "Positive" or "Negative".
Provide **only one word** as the response — either Positive or Negative.

Review: {review_text}
"""

# Function to call Claude via Bedrock and predict sentiment
def zero_shot_sentiment(review_text):
    prompt = prompt_template.format(review_text=review_text)

    request_payload = {
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": 10,
        "temperature": 0,
        "top_p": 0.9,
        "messages": [
            {"role": "user", "content": prompt}
        ]
    }

    response = client.invoke_model(
        modelId=model_id,
        contentType="application/json",
        accept="application/json",
        body=json.dumps(request_payload)
    )

    response_body = json.loads(response['body'].read().decode('utf-8'))

    # Correct parsing - read the first message from 'content' list
    predicted_label = response_body['content'][0]['text'].strip().lower()

    if "positive" in predicted_label:
        return 1
    elif "negative" in predicted_label:
        return 0
    else:
        raise ValueError(f"Unexpected response from Claude: {predicted_label}")

# Apply zero-shot sentiment analysis to the dataset (with progress bar)
selected_reviews['Zero_Shot_Prediction'] = [
    zero_shot_sentiment(review) for review in tqdm(selected_reviews['text'])
]

# Evaluate Zero-Shot Model Performance
y_true = selected_reviews['Sentiment']
y_pred = selected_reviews['Zero_Shot_Prediction']

print("Zero-Shot Learning Performance:")
print("Accuracy:", accuracy_score(y_true, y_pred))
print("Precision:", precision_score(y_true, y_pred))
print("Recall:", recall_score(y_true, y_pred))
print("F1 Score:", f1_score(y_true, y_pred))












  0%|          | 0/100 [00:00<?, ?it/s]









  1%|          | 1/100 [00:01<01:57,  1.19s/it]









  2%|▏         | 2/100 [00:01<01:07,  1.44it/s]









  3%|▎         | 3/100 [00:01<00:54,  1.78it/s]









  4%|▍         | 4/100 [00:02<00:43,  2.19it/s]









  5%|▌         | 5/100 [00:02<00:39,  2.38it/s]









  6%|▌         | 6/100 [00:03<00:54,  1.72it/s]









  7%|▋         | 7/100 [00:03<00:47,  1.98it/s]









  8%|▊         | 8/100 [00:04<00:40,  2.29it/s]









  9%|▉         | 9/100 [00:04<00:37,  2.43it/s]









 10%|█         | 10/100 [00:04<00:33,  2.70it/s]









 11%|█         | 11/100 [00:05<00:32,  2.75it/s]









 12%|█▏        | 12/100 [00:05<00:31,  2.80it/s]









 13%|█▎        | 13/100 [00:05<00:28,  3.03it/s]









 14%|█▍        | 14/100 [00:06<00:27,  3.14it/s]









 15%|█▌        | 15/100 [00:07<00:49,  1.73it/s]









 16%|█▌        | 16/100 [00:07<00:48,  1.72it/s]









 17%|█▋        | 17/10

Zero-Shot Learning Performance:
Accuracy: 0.97
Precision: 0.9795918367346939
Recall: 0.96
F1 Score: 0.9696969696969697


In [36]:
# Code Cell 4 - Perform Sentiment Analysis Using Few-Shot Learning
# Define the few-shot prompt template
few_shot_prompt_template = """
You are an expert sentiment analyzer trained to classify restaurant reviews.
Below are examples of labeled reviews:

Example 1: "The food was amazing and the service was excellent!" -> Positive
Example 2: "The worst experience ever, the food was cold and tasteless." -> Negative
Example 3: "I loved the ambiance, the food was delightful." -> Positive
Example 4: "Not worth the price, very disappointing." -> Negative

Now analyze the following review and classify it as either "Positive" or "Negative".
Provide **only one word** as the response — either Positive or Negative.

Review: {review_text}
"""

# Function to call Claude via Bedrock for few-shot learning
def few_shot_sentiment(review_text):
    prompt = few_shot_prompt_template.format(review_text=review_text)

    request_payload = {
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": 10,
        "temperature": 0,
        "top_p": 0.9,
        "messages": [
            {"role": "user", "content": prompt}
        ]
    }

    response = client.invoke_model(
        modelId=model_id,
        contentType="application/json",
        accept="application/json",
        body=json.dumps(request_payload)
    )

    response_body = json.loads(response['body'].read().decode('utf-8'))
    predicted_label = response_body['content'][0]['text'].strip().lower()

    if "positive" in predicted_label:
        return 1
    elif "negative" in predicted_label:
        return 0
    else:
        raise ValueError(f"Unexpected response from Claude: {predicted_label}")

# Apply few-shot sentiment analysis to the dataset
selected_reviews['Few_Shot_Prediction'] = [
    few_shot_sentiment(review) for review in tqdm(selected_reviews['text'])
]

# Evaluate Few-Shot Model Performance
y_pred_few_shot = selected_reviews['Few_Shot_Prediction']

print("Few-Shot Learning Performance:")
print("Accuracy:", accuracy_score(y_true, y_pred_few_shot))
print("Precision:", precision_score(y_true, y_pred_few_shot))
print("Recall:", recall_score(y_true, y_pred_few_shot))
print("F1 Score:", f1_score(y_true, y_pred_few_shot))












  0%|          | 0/100 [00:00<?, ?it/s]









  1%|          | 1/100 [00:00<00:44,  2.23it/s]









  2%|▏         | 2/100 [00:00<00:41,  2.37it/s]









  3%|▎         | 3/100 [00:01<00:45,  2.13it/s]









  4%|▍         | 4/100 [00:02<00:50,  1.89it/s]









  5%|▌         | 5/100 [00:02<00:52,  1.80it/s]









  6%|▌         | 6/100 [00:03<00:49,  1.89it/s]









  7%|▋         | 7/100 [00:03<00:44,  2.07it/s]









  8%|▊         | 8/100 [00:03<00:43,  2.11it/s]









  9%|▉         | 9/100 [00:04<00:43,  2.11it/s]









 10%|█         | 10/100 [00:04<00:40,  2.22it/s]









 11%|█         | 11/100 [00:05<00:44,  2.02it/s]









 12%|█▏        | 12/100 [00:06<00:49,  1.79it/s]









 13%|█▎        | 13/100 [00:06<00:50,  1.72it/s]









 14%|█▍        | 14/100 [00:07<00:49,  1.75it/s]









 15%|█▌        | 15/100 [00:07<00:45,  1.86it/s]









 16%|█▌        | 16/100 [00:08<00:58,  1.43it/s]









 17%|█▋        | 17/10

Few-Shot Learning Performance:
Accuracy: 0.98
Precision: 0.98
Recall: 0.98
F1 Score: 0.98


In [44]:
# Code Cell 5 - Experiment with Multiple LLMs

import boto3
import json
from tqdm import tqdm
from sklearn.metrics import classification_report

# Initialize Bedrock client
client = boto3.client("bedrock-runtime", region_name="us-east-1")

# Define model IDs for Claude and LLaMA
claude_model_id = 'anthropic.claude-3-sonnet-20240229-v1:0'  # LLM1
llama_model_id = 'meta.llama3-70b-instruct-v1:0'  # LLM2

# Function to summarize a review using Claude
def summarize_with_claude(review_text):
    prompt = f'''
    Review: {review_text}
    
    summarize the given review.
    '''
    
    payload = {
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": 100,
        "temperature": 0.5,
        "messages": [
            {"role": "user", "content": prompt}
        ]
    }
    
    response = client.invoke_model(
        modelId=claude_model_id,
        contentType="application/json",
        accept="application/json",
        body=json.dumps(payload)
    )
    
    response_body = json.loads(response["body"].read())
    summary = response_body.get("content", [""])[0].get("text", "")
    return summary

# Function to extract likes and dislikes using LLaMA
def extract_likes_dislikes_with_llama(summary_text):
    prompt = f'''
    Summary: {summary_text}
    Now from the summary, list the things the reviewer likes and dislikes.
    '''
    
    formatted_prompt = f"""<|begin_of_text|><|start_header_id|>user<|end_header_id|>{prompt}<|eot_id|><|start_header_id|>assistant<|end_header_id|> """
    
    payload = {
        "prompt": formatted_prompt,
        "max_gen_len": 512,
        "temperature": 0.5
    }
    
    response = client.invoke_model(
        modelId=llama_model_id,
        contentType="application/json",
        accept="application/json",
        body=json.dumps(payload)
    )
    
    response_body = json.loads(response["body"].read())
    likes_dislikes = response_body.get("generation", "")
    return likes_dislikes

# Apply summarization and likes/dislikes extraction
selected_reviews['Summary_Claude'] = [
    summarize_with_claude(review) for review in tqdm(selected_reviews['text'], desc="Summarizing with Claude")
]

selected_reviews['Likes_Dislikes_LLaMA'] = [
    extract_likes_dislikes_with_llama(summary) for summary in tqdm(selected_reviews['Summary_Claude'], desc="Extracting Likes/Dislikes with LLaMA")
]

# Display outputs
print("\nSample Outputs:")
print(selected_reviews[['text', 'Summary_Claude', 'Likes_Dislikes_LLaMA']].head(10))















Summarizing with Claude:   0%|          | 0/100 [00:00<?, ?it/s]













Summarizing with Claude:   1%|          | 1/100 [00:03<06:13,  3.77s/it]













Summarizing with Claude:   2%|▏         | 2/100 [00:06<04:59,  3.06s/it]













Summarizing with Claude:   3%|▎         | 3/100 [00:09<05:18,  3.29s/it]













Summarizing with Claude:   4%|▍         | 4/100 [00:11<04:29,  2.81s/it]













Summarizing with Claude:   5%|▌         | 5/100 [00:14<04:24,  2.78s/it]













Summarizing with Claude:   6%|▌         | 6/100 [00:18<04:40,  2.98s/it]













Summarizing with Claude:   7%|▋         | 7/100 [00:20<04:18,  2.77s/it]













Summarizing with Claude:   8%|▊         | 8/100 [00:22<04:02,  2.64s/it]













Summarizing with Claude:   9%|▉         | 9/100 [00:25<04:08,  2.73s/it]













Summarizing with Claude:  10%|█         | 10/100 [00:28<04:08,  2.76s/it]













Summarizing with Claude:  11%|█         | 11/10


Sample Outputs:
                                                    text  \
24176  We had the restaurant to ourselves, which was ...   
46200  When my family can't decide where to eat, we o...   
12935  Our experience was 100% positive.  The omlette...   
860    Bright, cheerful, clean, friendly with a large...   
17966  I ordered takeout for two...the food was a com...   
25868  I saw pictures on Google and I was in the area...   
29509  The food is always good here, especially the t...   
42826  I ate the other day with my Family we feel ver...   
7126   Linner on the massive patio was excellent. All...   
14110  Best bbq in Tucson!! If you haven't been to ke...   

                                          Summary_Claude  \
24176  Here's a summary of the review:\n\nThe custome...   
46200  Here's a summary of the review:\n\nThe review ...   
12935  Here's a summary of the review:\n\nThe review ...   
860    Here's a summary of the review:\n\nThe reviewe...   
17966  Here's a summar

**Text Cell 6 - Discussion and Observations**

**Discussion and Observations**

1. Comparison of Zero-Shot vs. Few-Shot Learning Performance

Zero-Shot Learning:

Accuracy: 0.97
Precision: 0.98
Recall: 0.96
F1 Score: 0.97

Observation: Very high accuracy, but slightly lower recall suggests it may miss some positive or negative sentiments.
Few-Shot Learning:

Accuracy: 0.98
Precision: 0.98
Recall: 0.98
F1 Score: 0.98
Observation: The few-shot approach slightly outperformed zero-shot in all metrics, demonstrating the benefit of giving the model a few labeled examples.

2. Cases Where LLM Predictions Differ from Actual Labels
There were a few cases where the zero-shot model missed sentiments that the few-shot model correctly identified.
For example, a review with subtle sarcasm or a mixed sentiment might have been misclassified by the zero-shot approach but correctly identified using few-shot learning, which had context from examples.

3. Potential Reasons for Misclassifications
Lack of Context: Zero-shot models may struggle without specific examples.
Ambiguous Reviews: Reviews that are neither entirely positive nor negative may lead to confusion.
Prompt Limitations: The generic prompt for zero-shot learning might not have been nuanced enough.

4. Differences in Outputs from Various LLMs
Claude (Summarization): The summaries generated were quite repetitive ("Here's a summary of the review..."). This suggests the prompt might need rephrasing or the model may need a more specific instruction.
LLaMA (Likes/Dislikes): The output included formatting issues (<|end_header_id|>). This indicates the prompt formatting for LLaMA might need adjustment, especially the use of |start_header_id| and |end_header_id| tags.

**Overall Analysis**
Few-shot learning provided a performance boost due to the availability of example-based learning.
The prompt design for both LLMs needs improvement to avoid generic and placeholder responses.
LLaMA's output formatting needs refinement to avoid displaying special tokens and to ensure it outputs clear and structured likes and dislikes.

**Recommendations for Improvement**

- Enhance Prompt Engineering: Provide more concrete and structured examples for the few-shot learning model.
                                                                         
- Refine Prompts for Claude and LLaMA: Avoid boilerplate language and use more natural, directive prompts.
                                                                         
- Pre-process Model Outputs: Strip out unwanted tokens (|start_header_id|, |end_header_id|) from LLaMA's output.
    
- Experiment with Temperature and Max Tokens: A lower temperature and properly set max_tokens could lead to more concise and relevant outputs.
    

**Text Cell 7 - Acknowledge if you have used any GenAI tools or collaborated with others.**

- I used Generative AI tools, specifically Amazon Bedrock’s Claude 3 Sonnet and LLaMA 3, for sentiment analysis, summarization, and extracting insights from reviews. Furthermore, I confirm that I worked independently on this assignment and did not collaborate with any classmates or external individuals.

In [5]:
#HTLM output - Convert the notebook to an HTML file ensuring all outputs are visible.
!pip install jupyter
!pip install nbconvert
!jupyter nbconvert "LA6_Singh_Balbir" --to html

[NbConvertApp] Converting notebook LA6_Singh_Balbir.ipynb to html
[NbConvertApp] Writing 371207 bytes to LA6_Singh_Balbir.html
